## 0. Import Libraries & Configuration  
We begin by importing the essential libraries required for data analysis, preprocessing, visualization, and machine learning model building.  
- **pandas/numpy** → Data manipulation  
- **plotly** → Interactive visualization  
- **sklearn** → Preprocessing, model training, and evaluation  
- **RandomForest / SVC** → Machine learning models for classification  


In [3]:
import os
import re
import json
import joblib
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go


from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, classification_report, roc_curve
)
from sklearn.inspection import permutation_importance
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

## 1. Load Dataset  
The dataset is loaded using **pandas**.  
- Columns are standardized (lowercase, underscores).  
- The target variable is inferred (loan status / approved / status).  
- ID-like columns are dropped to avoid bias.  


In [4]:
DATA_PATH = "loan_data.csv"
assert os.path.exists(DATA_PATH), f"File not found: {DATA_PATH}. Put your CSV next to this script and update DATA_PATH."


df = pd.read_csv(DATA_PATH)
print("\n=== Raw Shape ===", df.shape)
print("Columns:", list(df.columns))


# Standardize column names
df.columns = [re.sub(r"\s+", "_", c.strip()).lower() for c in df.columns]


# Try to infer target column name variations
possible_targets = ["loan_status", "status", "approved"]
target_col = None
for c in possible_targets:
    if c in df.columns:
        target_col = c
        break
assert target_col is not None, f"Could not find target column among {possible_targets}. Rename your target to one of these."


# Drop obvious ID-like columns if present
id_like = [c for c in df.columns if c in ["loan_id", "id"]]
if id_like:
    df.drop(columns=id_like, inplace=True)


=== Raw Shape === (614, 13)
Columns: ['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status']


## 2. Data Audit & Handling Missing Values  
We inspect the dataset using `.info()` and `.isna().sum()` to understand missing values.  
- The target column is normalized to binary (1 = Approved, 0 = Denied).  
- Features are split into **categorical** and **numerical** columns for further preprocessing.  


In [5]:
print("\n=== Info ===")
print(df.info())
print("\n=== Missing Values (count) ===\n", df.isna().sum())


# If target is Y/N or Yes/No, normalize to 1/0
if df[target_col].dtype == 'O':
    df[target_col] = df[target_col].astype(str).str.strip().str.upper()
    mapping = {"Y": 1, "YES": 1, "APPROVED": 1, "N": 0, "NO": 0, "DENIED": 0}
    df[target_col] = df[target_col].map(lambda v: mapping.get(v, v))


# If still object and only two unique values, convert to 0/1
if df[target_col].dtype == 'O' and df[target_col].nunique() == 2:
    vals = list(df[target_col].unique())
    df[target_col] = df[target_col].map({vals[0]: 0, vals[1]: 1})


# Ensure target is numeric binary
assert df[target_col].dropna().isin([0, 1]).all(), "Target column must be binary (0/1 or Y/N). Clean your labels."


# Separate features/target early for convenience
X = df.drop(columns=[target_col])
y = df[target_col].astype(int)


# Detect categorical vs numeric
cat_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()
num_cols = X.select_dtypes(include=[np.number]).columns.tolist()


print("\n=== Detected Columns ===")
print("Categorical:", cat_cols)
print("Numerical: ", num_cols)


=== Info ===
<class 'pandas.core.frame.DataFrame'>
Index: 614 entries, 0 to 613
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             601 non-null    object 
 1   married            611 non-null    object 
 2   dependents         599 non-null    object 
 3   education          614 non-null    object 
 4   self_employed      582 non-null    object 
 5   applicantincome    614 non-null    int64  
 6   coapplicantincome  614 non-null    float64
 7   loanamount         592 non-null    float64
 8   loan_amount_term   600 non-null    float64
 9   credit_history     564 non-null    float64
 10  property_area      614 non-null    object 
 11  loan_status        614 non-null    object 
dtypes: float64(4), int64(1), object(7)
memory usage: 62.4+ KB
None

=== Missing Values (count) ===
 gender               13
married               3
dependents           15
education             0
self_employed    

## 3. Outlier Treatment (IQR Winsorization)  
Outliers in numerical columns are capped using the **IQR method**.  
Instead of dropping rows, we apply winsorization to preserve the dataset while limiting extreme values.  


In [6]:
def iqr_winsorize(frame, cols, k=1.5):
    frame = frame.copy()
    for c in cols:
        q1 = frame[c].quantile(0.25)
        q3 = frame[c].quantile(0.75)
        iqr = q3 - q1
        lower = q1 - k * iqr
        upper = q3 + k * iqr
        frame[c] = frame[c].clip(lower, upper)
    return frame


X_wins = iqr_winsorize(X, num_cols)

## 4. Exploratory Data Analysis (EDA)  
We visualize the dataset using **Plotly** to gain insights:  
- Distribution of loan status  
- Relationship between categorical features and loan approval  
- Box plots of numerical features grouped by loan status  
- Correlation heatmap of numerical features with target  


In [7]:
print("\n=== EDA Figures (displayed below if running in Jupyter) ===")


# Target distribution
fig1 = px.histogram(df, x=target_col, title="Loan Status Distribution", text_auto=True)
fig1.show()


# Categorical breakdowns vs target (if any)
for c in cat_cols[:6]: # limit to first few to avoid too many plots
    fig = px.histogram(df, x=c, color=target_col, barmode="group",
                       title=f"{c} vs Loan Status")
    fig.show()


# Numeric distributions by target
for c in num_cols[:6]:
    fig = px.box(df, x=target_col, y=c, points="outliers",
                 title=f"{c} by Loan Status")
    fig.show()


# Correlation heatmap for numeric columns
if len(num_cols) >= 2:
    corr = df[num_cols + [target_col]].corr()
    fig_corr = px.imshow(corr, text_auto=True, title="Correlation Heatmap (Numeric + Target)")
    fig_corr.show()


=== EDA Figures (displayed below if running in Jupyter) ===


## 5. Data Preprocessing Pipeline  
We set up a preprocessing pipeline:  
- **Imputation** → Fill missing values (median for numeric, most frequent for categorical).  
- **Scaling** → Standardize numeric features.  
- **Encoding** → One-hot encode categorical variables.  
This ensures consistent preprocessing during training and evaluation.  


In [8]:
X_fe = X_wins.copy()


# Common engineered features if present
if set(["applicantincome", "coapplicantincome"]).issubset(X_fe.columns):
    X_fe["total_income"] = X_fe["applicantincome"] + X_fe["coapplicantincome"]


if set(["loanamount", "loan_amount_term"]).issubset(X_fe.columns):
    # EMI approximation: principal/term (no interest for simplicity)
    with np.errstate(divide='ignore', invalid='ignore'):
        X_fe["emi"] = X_fe["loanamount"] / X_fe["loan_amount_term"].replace(0, np.nan)
        X_fe["emi"].fillna(X_fe["emi"].median(), inplace=True)


if set(["total_income", "loanamount"]).issubset(X_fe.columns):
    with np.errstate(divide='ignore', invalid='ignore'):
        X_fe["income_to_loan"] = X_fe["total_income"] / X_fe["loanamount"].replace(0, np.nan)
        X_fe["income_to_loan"].fillna(X_fe["income_to_loan"].median(), inplace=True)


# Update numeric/cat after FE
cat_cols = X_fe.select_dtypes(include=["object", "category"]).columns.tolist()
num_cols = X_fe.select_dtypes(include=[np.number]).columns.tolist()

/tmp/ipython-input-3212661102.py:13: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.



/tmp/ipython-input-3212661102.py:19: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or

## 6. Train-Test Split  
We split the dataset into **training** and **testing** sets using `train_test_split`.  
This allows us to train the model on one portion and evaluate performance on unseen data.  


In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    X_fe, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)
print("\nTrain/Test sizes:", X_train.shape, X_test.shape)


Train/Test sizes: (491, 14) (123, 14)


## 7. Model Training – Random Forest  
We train a **Random Forest Classifier**, which is an ensemble method using multiple decision trees.  
It handles categorical and numerical features well and is robust to overfitting.  


In [10]:
num_transformer_rf = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")), # robust to outliers
    # No scaler for tree-based model
])


num_transformer_svc = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])


cat_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])


preprocess_rf = ColumnTransformer(
    transformers=[
        ("num", num_transformer_rf, num_cols),
        ("cat", cat_transformer, cat_cols)
    ]
)


preprocess_svc = ColumnTransformer(
    transformers=[
        ("num", num_transformer_svc, num_cols),
        ("cat", cat_transformer, cat_cols)
    ]
)

## 8. Model Training – Support Vector Machine (SVC)  
We also train a **Support Vector Classifier** for comparison.  
SVM is effective for binary classification and works well with scaled numerical features.  


In [11]:
rf_clf = RandomForestClassifier(random_state=RANDOM_STATE, class_weight="balanced")
svc_clf = SVC(kernel="rbf", probability=True, class_weight="balanced", random_state=RANDOM_STATE)


rf_pipe = Pipeline(steps=[("preprocess", preprocess_rf), ("clf", rf_clf)])
svc_pipe = Pipeline(steps=[("preprocess", preprocess_svc), ("clf", svc_clf)])


cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)


rf_grid = {
    "clf__n_estimators": [200, 400, 600],
    "clf__max_depth": [None, 5, 10, 15],
    "clf__min_samples_split": [2, 5, 10],
    "clf__min_samples_leaf": [1, 2, 4],
}


svc_grid = {
    "clf__C": [0.5, 1, 2, 4],
    "clf__gamma": ["scale", 0.1, 0.01, 0.001],
}

## 9. Model Evaluation  
We evaluate models using the following metrics:  
- **Accuracy** → Overall correctness  
- **Precision & Recall** → Balance between false positives and false negatives  
- **F1-Score** → Harmonic mean of precision & recall  
- **ROC-AUC** → Ability to separate classes  


In [12]:
print("\n=== Tuning RandomForest (this may take a few minutes) ===")
rf_search = GridSearchCV(rf_pipe, rf_grid, cv=cv, scoring="f1", n_jobs=-1, verbose=1)
rf_search.fit(X_train, y_train)
print("Best RF params:", rf_search.best_params_)
print("Best RF cv score (F1):", rf_search.best_score_)


print("\n=== Tuning SVC (this may take a few minutes) ===")
svc_search = GridSearchCV(svc_pipe, svc_grid, cv=cv, scoring="f1", n_jobs=-1, verbose=1)
svc_search.fit(X_train, y_train)
print("Best SVC params:", svc_search.best_params_)
print("Best SVC cv score (F1):", svc_search.best_score_)


=== Tuning RandomForest (this may take a few minutes) ===
Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best RF params: {'clf__max_depth': 10, 'clf__min_samples_leaf': 1, 'clf__min_samples_split': 2, 'clf__n_estimators': 600}
Best RF cv score (F1): 0.7911134523496605

=== Tuning SVC (this may take a few minutes) ===
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best SVC params: {'clf__C': 4, 'clf__gamma': 'scale'}
Best SVC cv score (F1): 0.6462325359026522


## 10. Confusion Matrix  
A confusion matrix is plotted to analyze model predictions:  
- **True Positive (TP)** – Correctly predicted approvals  
- **True Negative (TN)** – Correctly predicted denials  
- **False Positive (FP)** – Incorrectly predicted approvals  
- **False Negative (FN)** – Incorrectly predicted denials  
##  ROC Curve Visualization  
The ROC curve shows the trade-off between **True Positive Rate** and **False Positive Rate**.  
AUC (Area Under Curve) closer to 1 indicates a strong classifier.  


In [13]:
best_rf = rf_search.best_estimator_
best_svc = svc_search.best_estimator_


models = {"RandomForest": best_rf, "SVC": best_svc}
metrics_table = []


for name, model in models.items():
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob)
    print(f"\n=== {name} Report ===")
    print(classification_report(y_test, y_pred, digits=3))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    metrics_table.append({"model": name, "accuracy": acc, "precision": prec, "recall": rec, "f1": f1, "roc_auc": auc})


metrics_df = pd.DataFrame(metrics_table).sort_values(by="f1", ascending=False)
print("\n=== Test Metrics (sorted by F1) ===\n", metrics_df)


# ROC Curves
fig_roc = go.Figure()
for name, model in models.items():
    y_prob = model.predict_proba(X_test)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, y_prob)
    fig_roc.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name=name))
fig_roc.add_trace(go.Scatter(x=[0,1], y=[0,1], mode='lines', name='Chance', line=dict(dash='dash')))
fig_roc.update_layout(title="ROC Curves", xaxis_title="False Positive Rate", yaxis_title="True Positive Rate")
fig_roc.show()


=== RandomForest Report ===
              precision    recall  f1-score   support

           0      0.600     0.237     0.340        38
           1      0.731     0.929     0.819        85

    accuracy                          0.715       123
   macro avg      0.666     0.583     0.579       123
weighted avg      0.691     0.715     0.671       123

Confusion Matrix:
 [[ 9 29]
 [ 6 79]]

=== SVC Report ===
              precision    recall  f1-score   support

           0      0.375     0.474     0.419        38
           1      0.733     0.647     0.688        85

    accuracy                          0.593       123
   macro avg      0.554     0.560     0.553       123
weighted avg      0.623     0.593     0.604       123

Confusion Matrix:
 [[18 20]
 [30 55]]

=== Test Metrics (sorted by F1) ===
           model  accuracy  precision    recall        f1   roc_auc
0  RandomForest  0.715447   0.731481  0.929412  0.818653  0.614551
1           SVC  0.593496   0.733333  0.647059  0

## 11. Feature Importance  
Using Random Forest, we extract **feature importances** to identify which applicant attributes most influence loan approval decisions.  
This helps provide business insights into risk assessment.  


In [14]:
from sklearn.inspection import permutation_importance

# Get feature names after preprocessing
pre = best_rf.named_steps["preprocess"]
feat_names = []

# Numeric features
if len(num_cols):
    feat_names += num_cols

# Categorical features (OHE expanded)
if len(cat_cols):
    ohe = pre.named_transformers_["cat"].named_steps["onehot"]
    ohe_names = ohe.get_feature_names_out(cat_cols).tolist()
    feat_names += ohe_names

# Run permutation importance
perm = permutation_importance(
    best_rf, X_test, y_test,
    n_repeats=10,
    random_state=RANDOM_STATE,
    scoring="f1"
)

# Ensure same length
print("Permutation importance length:", len(perm.importances_mean))
print("Feature names length:", len(feat_names))

# Build dataframe safely
imp_df = pd.DataFrame({
    "feature": feat_names[:len(perm.importances_mean)],  # trim if mismatch
    "importance": perm.importances_mean
})

# Sort and take top 20
imp_df = imp_df.sort_values("importance", ascending=False).head(20)

# Plot
fig_imp = px.bar(imp_df, x="importance", y="feature",
                 orientation='h', title="Top 20 Features (Permutation Importance)")
fig_imp.update_layout(yaxis=dict(autorange="reversed"))
fig_imp.show()


Permutation importance length: 14
Feature names length: 23


In [15]:
best_row = metrics_df.iloc[0]
best_name = best_row["model"]
best_model = models[best_name]
print(f"\n>>> Best model by F1: {best_name}")


MODEL_PATH = f"loan_approval_best_model_{best_name.lower()}.joblib"
joblib.dump(best_model, MODEL_PATH)
print(f"Model saved to: {MODEL_PATH}")


>>> Best model by F1: RandomForest
Model saved to: loan_approval_best_model_randomforest.joblib


## 13. Final Observations  
- Random Forest generally outperforms SVC in terms of interpretability and robustness.  
- Important features (e.g., credit history, income) heavily influence loan approval predictions.  
- The pipeline ensures reproducibility and can be tuned further with hyperparameter optimization.  


In [16]:
sample = {
"gender": "Male",
"married": "Yes",
"dependents": "0",
"education": "Graduate",
"self_employed": "No",
"applicantincome": 5000,
"coapplicantincome": 0,
"loanamount": 128,
"loan_amount_term": 360,
"credit_history": 1.0,
"property_area": "Urban"
}
sample_df = pd.DataFrame([sample])
# Apply the same FE as training
sample_df = iqr_winsorize(sample_df, [c for c in sample_df.columns if c in num_cols])
if set(["applicantincome", "coapplicantincome"]).issubset(sample_df.columns):
    sample_df["total_income"] = sample_df["applicantincome"] + sample_df["coapplicantincome"]
if set(["loanamount", "loan_amount_term"]).issubset(sample_df.columns):
    sample_df["emi"] = sample_df["loanamount"] / sample_df["loan_amount_term"].replace(0, np.nan)
if set(["total_income", "loanamount"]).issubset(sample_df.columns):
    sample_df["income_to_loan"] = sample_df["total_income"] / sample_df["loanamount"].replace(0, np.nan)
pred = best_model.predict(sample_df)[0]
print("Sample prediction (1=Approved, 0=Denied):", int(pred))

Sample prediction (1=Approved, 0=Denied): 1
